In [1]:
import spglib 

# make a dict of space group info
# key: international_short
# value: space group info
# space group info: (number, international_short, international_full, international, schoenflies, hall)
space_group_info_dict = {}
for i in range(1, 531):
    space_group_info_dict[spglib.get_spacegroup_type(i)["international_short"]] = spglib.get_spacegroup_type(i)
    


/Users/chenxili/miniconda/envs/vlx/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpaceGroupType['international_short']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


In [2]:
space_group_info_dict

{'P1': SpaceGroupType(number=1, international_short='P1', international_full='P 1', international='P 1', schoenflies='C1^1', hall_number=1, hall_symbol='P 1', choice='', pointgroup_international='1', pointgroup_schoenflies='C1', arithmetic_crystal_class_number=1, arithmetic_crystal_class_symbol='1P'),
 'P-1': SpaceGroupType(number=2, international_short='P-1', international_full='P -1', international='P -1', schoenflies='Ci^1', hall_number=2, hall_symbol='-P 1', choice='', pointgroup_international='-1', pointgroup_schoenflies='Ci', arithmetic_crystal_class_number=2, arithmetic_crystal_class_symbol='-1P'),
 'P2': SpaceGroupType(number=3, international_short='P2', international_full='P 2 1 1', international='P 2 = P 2 1 1', schoenflies='C2^1', hall_number=5, hall_symbol='P 2x', choice='a', pointgroup_international='2', pointgroup_schoenflies='C2', arithmetic_crystal_class_number=3, arithmetic_crystal_class_symbol='2P'),
 'P2_1': SpaceGroupType(number=4, international_short='P2_1', intern

In [3]:
with open('/Users/chenxili/GitHub/vlx_mofbuilder/data/RCSRnets-2019-06-01.cgd') as f:
    lines = f.readlines()
#extract the space group info from the cgd file, start with CRYSTAL end with END
starts = []
ends = []
for i, line in enumerate(lines):
    if "CRYSTAL" in line or "crystal" in line:
        if "#" in line:
            continue
        starts.append(i)
    if "END" in line or "end" in line:
        if "#" in line:
            continue
        end = i
        ends.append(i)
#check if the number of starts and ends are the same
if len(starts) != len(ends):
    print("Error: the number of starts and ends are not the same")

#extract the space group info from the cgd file
crystal_infos = []
for i in range(len(starts)):
    crystal_info = []
    if ends[i] - starts[i] < 1:
        print("Error: the crystal info is not complete",starts[i],ends[i],lines[starts[i]:ends[i]+1])
        continue

    for j in range(starts[i], ends[i]+1):
        crystal_info.append(lines[j])
    crystal_infos.append(crystal_info)



          

In [7]:
import re
def similar_group_name(group):
    #make first letter uppercase
    if ":" in group:
        group = group.split(":")[0]
    return re.sub(r'[^a-zA-Z0-9]', '', group)

#https://cci.lbl.gov/sginfo/hall_symbols.html
#2d https://pscf.readthedocs.io/en/latest/groups.html
def fetch_group_number(group):
    numbers = []
    if group =="C12/m1":
        numbers = [71]
        return numbers
    if group =="Cmca":
        numbers = [304] #? a/e
        return numbers
    if group =="C2mm":
        numbers = [188]
        return numbers
    if group =="P121/n1":
        numbers = [89]
        return numbers

    if group =="Ccca:2":
        numbers = [325] #? a/e
        return numbers
    if group =="Cmma":
        numbers = [317] #? a/e
        return numbers
    if group =="C12/c1":
        numbers = [107]
        return numbers
    if group =="I12/a1":
        numbers = [107]
        return numbers
    if group =="I12/m1":
        numbers = [71]
        return numbers
    if group =="P121/c1":
        numbers = [89]
        return numbers
    #if group =="llw-z":
    #    numbers = [196]
    #    return numbers
    if group =="R-3:H":
        numbers = [148]
        return numbers
    if group =="R3m:H":
        numbers = [160]
        return numbers

    if group =="R3c:H":
        numbers = [161]
        return numbers

    if group =="R-3m:H":
        numbers = [166]
        return numbers
    if group =="R-3c:H":
        numbers = [167]
        return numbers

    if group[0] =='p':
        print("2d group")
        return [-1]
    if group[0] =='c':
        print("2d group")
        return [-1]

    for key in space_group_info_dict.keys():
        if group == key:
            numbers=[space_group_info_dict[key].hall_number]
            return numbers
        elif similar_group_name(group) == similar_group_name(key):
            #print('hall number:', space_group_info_dict[key])
            numbers.append(space_group_info_dict[key].hall_number)
    if len(numbers) >1:
        print()
        print('more than one space group found',group,numbers)
    elif len(numbers) == 0:
        print('*'*10)
        print('no space group found',group)
        print('*'*10)
    return numbers
    


#convert group name to hall number 
#looking up the space group number from the space group name
#check the group name is in the space group info dict keys, or similar to the space group name
#extract all number and letter from the group name, use re to replace all non letter non number characters with ''
#then check if the group name is in the space group info dict keys

In [9]:
for net in crystal_infos:
    if len(net) <2:
        print(net)
        continue
    #extract name, group, cell, nodes, and edge centers from net file
    name = net[1].strip('\n').split()[1]
    group = net[2].strip('\n').split()[1]
    cell = net[3].strip('\n').split()[1:]
    nodes = []
    edge_centers = []
    for line in net[4:]:
        line = line.strip('\n')
        if 'NODE' in line:
            nodes.append(line.split()[1:])
        if 'EDGE_CENTER' in line:
            edge_centers.append(line.split()[2:])
    if len(nodes) <2: 
        pass
    #    print('ditopic network')
    #elif len(nodes) == 2: 
    #    print('multitopic network')
    else:
        continue
    #    print('cannot handle this network')
    group_number = fetch_group_number(group)[0]
    if group_number == -1:
        continue
    symmetry_operations = spglib.get_symmetry_from_database(group_number)
    #print(name,group,group_number,cell,len(nodes),nodes,edge_centers)
        

2d group
2d group
2d group
2d group
2d group
2d group
2d group
2d group
2d group
2d group
2d group


In [ ]:
fetch_group_number('Fm-3m')[0] #523 as hall_number
dataset = spglib.get_symmetry_from_database(523)
rotations = dataset["rotations"]
translations = dataset["translations"]
len(rotations), translations

(192,
 array([[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 

In [ ]:
# Apply symmetry operations to the nodes and edge centers
import numpy as np
import spglib

def apply_symmetry_operations(lattice, positions, space_group_number):
    """
    Apply symmetry operations to positions using a standardized primitive cell.
    """
    # Convert to primitive cell using spglib
    standardized_cell = spglib.standardize_cell((lattice, positions, [1] * len(positions)), to_primitive=True, no_idealize=True)

    # Extract the primitive lattice, positions, and types
    primitive_lattice, primitive_positions, _ = standardized_cell

    # Get symmetry operations for the space group
    dataset = spglib.get_symmetry_from_database(space_group_number)
    rotations = dataset["rotations"]
    translations = dataset["translations"]

    # Apply symmetry operations
    transformed_positions = set()

    for rot, trans in zip(rotations, translations):
        for pos in primitive_positions:
            new_pos = np.dot(rot, pos) + trans  # Apply rotation and translation
            new_pos = np.mod(new_pos, 1)  # Ensure periodic boundary conditions
            transformed_positions.add(tuple(new_pos))  # Store unique positions

    # Convert to numpy array and sort for consistency
    transformed_positions = np.array(sorted(transformed_positions))

    return transformed_positions


# **🔹 Define Input Parameters**
space_group_number = 529  # Fm-3m

# **Lattice parameters (converted to a 3×3 matrix)**
a = 1.41421
lattice = np.array([
    [a, 0, 0],
    [0, a, 0],
    [0, 0, a]
])

# **Initial node and edge positions (fractional coordinates)**
node_positions = np.array([
    [0.00000, 0.00000, 0.00000],  # Vertex
 # Face center
])

edge_positions = np.array([
    [0.25000, 0.25000, 0.00000],  # Edge center
])

# **🔹 Process nodes and edges separately**
symmetrized_nodes = apply_symmetry_operations(lattice, node_positions, space_group_number)
symmetrized_edges = apply_symmetry_operations(lattice, edge_positions, space_group_number)

# **🔹 Print results in the required format**
print("Generated Symmetrized Positions:\n")

# Print nodes
for i, pos in enumerate(symmetrized_nodes, start=1):
    print(f"V{i:<3}  V   {pos[0]:.5f}  {pos[1]:.5f}  {pos[2]:.5f}")

# Print edges
for i, pos in enumerate(symmetrized_edges, start=1):
    print(f"E{i:<3}  E   {pos[0]:.5f}  {pos[1]:.5f}  {pos[2]:.5f}")



Generated Symmetrized Positions:

V1    V   0.00000  0.00000  0.00000
V2    V   0.50000  0.50000  0.50000
E1    E   0.00000  0.25000  0.25000
E2    E   0.00000  0.25000  0.75000
E3    E   0.00000  0.75000  0.25000
E4    E   0.00000  0.75000  0.75000
E5    E   0.25000  0.00000  0.25000
E6    E   0.25000  0.00000  0.75000
E7    E   0.25000  0.25000  0.00000
E8    E   0.25000  0.25000  0.50000
E9    E   0.25000  0.50000  0.25000
E10   E   0.25000  0.50000  0.75000
E11   E   0.25000  0.75000  0.00000
E12   E   0.25000  0.75000  0.50000
E13   E   0.50000  0.25000  0.25000
E14   E   0.50000  0.25000  0.75000
E15   E   0.50000  0.75000  0.25000
E16   E   0.50000  0.75000  0.75000
E17   E   0.75000  0.00000  0.25000
E18   E   0.75000  0.00000  0.75000
E19   E   0.75000  0.25000  0.00000
E20   E   0.75000  0.25000  0.50000
E21   E   0.75000  0.50000  0.25000
E22   E   0.75000  0.50000  0.75000
E23   E   0.75000  0.75000  0.00000
E24   E   0.75000  0.75000  0.50000


In [ ]:
count = 0
for key in space_group_info_dict.keys():
    if group == key or similar_group_name(group) == similar_group_name(key):
        print('hall number:', space_group_info_dict[key])
        count += 1
if count >1:
    print('more than one space group found')
elif count == 0:
    print('no space group found')
        

no space group found
